In [4]:
%store -r  X_train

%store -r y_train
%store -r X_test
%store -r y_test

In [5]:
X_train=X_train.replace({True: 1, False: 0})
 

/var/folders/gq/789ffg0j3_s7scj27n_mdkj40000gn/T/ipykernel_21662/4021811479.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train=X_train.replace({True: 1, False: 0})


In [6]:
import pandas as pd
import numpy as np

In [7]:
dummies=pd.get_dummies(X_train['city'],drop_first=True)
X_train = X_train.drop(columns='city', axis=1)
X_train=X_train.merge(dummies,left_index=True,right_index=True)
X_train.head()

,id,date,dcoilwtico,transactions,store_nbr,item_nbr,cluster,perishable,class,year,...,Loja,Machala,Manta,Playas,Puyo,Quevedo,Quito,Riobamba,Salinas,Santo Domingo
5243140,65645802,2015-12-22,36.12,3138,34,1696041,6,1,2030,2015,...,False,False,False,False,False,False,False,False,False,False
11887704,116882650,2017-05-26,49.58,1411,28,454414,10,0,1034,2017,...,False,False,False,False,False,False,False,False,False,False
6456792,76894755,2016-04-20,42.72,2798,48,1501525,14,1,2004,2016,...,False,False,False,False,False,False,True,False,False,False
1243272,29081179,2014-08-15,97.30,1800,6,1239854,13,1,2420,2014,...,False,False,False,False,False,False,True,False,False,False
9023413,88068123,2016-08-15,45.72,1295,14,514242,7,0,1026,2016,...,False,False,False,False,False,False,False,True,False,False


In [8]:
corr_matrix=pd.merge(X_train,y_train,left_index=True, right_index=True).select_dtypes(['int','float']).corr()

In [9]:
corr_matrix.loc[corr_matrix['unit_sales']>0.02,'unit_sales']

transactions              0.068334
perishable                0.026199
onpromotion_True          0.030458
family_BEVERAGES          0.027349
family_POULTRY            0.020923
family_PRODUCE            0.045515
state_Pichincha           0.037458
logscaled_transactions    0.065527
unit_sales                1.000000
Name: unit_sales, dtype: float64

In [10]:
x=X_train[['transactions','perishable','onpromotion_True','family_BEVERAGES','family_POULTRY','family_PRODUCE','state_Pichincha','Quito']]
y=y_train

In [11]:
# Initialize the model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(x.values, y.values.ravel())

RandomForestRegressor(random_state=42)

In [12]:
dummies=pd.get_dummies(X_test['city'],drop_first=True)
X_test = X_test.drop(columns='city', axis=1)
X_test=X_test.merge(dummies,left_index=True,right_index=True)

In [13]:
y_pred=rf_model.predict(X_test[['transactions','perishable','onpromotion_True','family_BEVERAGES','family_POULTRY','family_PRODUCE','state_Pichincha','Quito']])

/Users/catherineliu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [14]:
 
print(f"root mse: {np.sqrt(mean_squared_error(y_test.values, y_pred))}")



root mse: 31.04712968021568


In [15]:
r2_score(y_test.values, y_pred)

0.027950689793181493

In [17]:
x.head()

,transactions,perishable,onpromotion_True,family_BEVERAGES,family_POULTRY,family_PRODUCE,state_Pichincha,Quito
5243140,3138,1,1,0,0,1,0,False
11887704,1411,0,0,0,0,0,0,False
6456792,2798,1,1,0,0,1,1,True
1243272,1800,1,1,0,1,0,1,True
9023413,1295,0,0,0,0,0,0,False


In [18]:
from sklearn.linear_model import LinearRegression,SGDRegressor,ElasticNet,Ridge

from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
# Select features based on importance
model = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=0)  # alpha is the regularization parameter, l1_ratio is the mixing parameter
model.fit(x.values, y.values.ravel())

y_pred_elastic = model.predict(X_test[['transactions','perishable','onpromotion_True','family_BEVERAGES','family_POULTRY','family_PRODUCE','state_Pichincha','Quito']].copy())






/Users/catherineliu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but ElasticNet was fitted without feature names
  warnings.warn(


In [19]:
# Calculate performance metrics
mse=np.sqrt(mean_squared_error(y_test.copy(), y_pred_elastic))
r2 = r2_score(y_test.copy(), y_pred_elastic)
print(f"root mse: {mse}")
print(f"R^2 Score: {r2}")

root mse: 31.31083032517557
R^2 Score: 0.011368248203655784


In [24]:
model = LinearRegression()
model.fit(x.values, y.values.ravel())
predictions = model.predict(X_test[['transactions','perishable','onpromotion_True','family_BEVERAGES','family_POULTRY','family_PRODUCE','state_Pichincha','Quito']].copy())


/Users/catherineliu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [25]:
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)


Coefficients: [ 2.94181934e-03  5.33315962e-01  4.44778366e+00  4.90909155e+00
  8.62404041e+00  8.54505797e+00  9.56963442e-01 -1.13815268e+00]
Intercept: 1.0019586747200169


In [26]:
# Calculate performance metrics
mse=np.sqrt(mean_squared_error(y_test.copy(), predictions))
r2 = r2_score(y_test.copy(), predictions)
print(f"root mse: {mse}")
print(f"R^2 Score: {r2}")

root mse: 31.14860946636408
R^2 Score: 0.0215858780627044
